# What is about 

Some examples to wrap up your mind how blend is working - may be useful for pedagogical reasons.

Diffrent fold splitings are consiered and cv-predictions are averaged (blended) - in most cases we see improvement of scores. 
Plots showing dependence on number of blended predictions are presented.
(Typically we see some saturation of quality after 5-10-20 blends). 
Also we consider several variants with different fold numbers. 

In different versions of the notebook we take different targets from MoA competition and examine them. 


V5 hsp_inhibitor - Logreg - results slighly contradictory - for 2 - not mononotic with respect to blend number ,  
3-folds blending seems to be better than 5 and 10 fold blending


V4 raf_inhibitor - LogReg - blend on 3-folds is better than on 2-folds, 5-folds a little worse than 3-fold, 10-fold is best, but convergence is slower


V3 some examples with logreg on target 'dopamine_receptor_antagonist' which is very little predictable. 
We see that blend can work very good on such target - killing noise and going to constant

V2 Blend up to 100 iterations  (model - LGB)

V1 First experiments - see that up to 5 iterations blend improves  (model - LGB)


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import datetime
import time

import matplotlib.pyplot as plt
import seaborn as sns

import time
df = pd.read_csv('/kaggle/input/lish-moa/train_features.csv',index_col = 0)  
df0 = df.copy()
df['cp_type'] = df['cp_type'].map({'trt_cp':1.0, 'ctl_vehicle':1.0}) # Forget about control group  
df['cp_dose'] = df['cp_dose'].map({'D1':0.0, 'D2':1.0})
df['cp_time'] = df['cp_time'].map({24:0.0, 48: .5 , 72:1.0})
X = df.copy()
X_save = df.copy()
df_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv',index_col = 0)
df_test['cp_type'] = df_test['cp_type'].map({'trt_cp':1.0, 'ctl_vehicle':0.0})
df_test['cp_dose'] = df_test['cp_dose'].map({'D1':0.0, 'D2':1.0})
df_test['cp_time'] = df_test['cp_time'].map({24:0.0, 48: .5 , 72:1.0})

y = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv',index_col = 0 )
y_save = y.copy()
print(y.iloc[:3,:2])
df

In [ ]:
y_save.sum(axis = 0 ).sort_values(ascending = False)

In [ ]:
y = y_save['cyclooxygenase_inhibitor']
print( y.sum(), len(y), y.sum()/len(y), ) 

y = y_save['dopamine_receptor_antagonist']
print( y.sum(), len(y), y.sum()/len(y), ) 

y = y_save['raf_inhibitor']
print( y.sum(), len(y), y.sum()/len(y), ) 


y = y_save['hsp_inhibitor']
print( y.sum(), len(y), y.sum()/len(y), ) 


In [ ]:
model = LogisticRegression(random_state=0)#,C = 0.001)


# 2 Folds Full data

In [ ]:
# X = X.iloc[:,:2]
X = X_save.copy()

In [ ]:
#sklearn.model_selection.cross_validate(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, 
# verbose=0, fit_params=None, pre_dispatch='2*n_jobs', return_train_score=False, return_estimator=False, error_score=nan)

model = LogisticRegression(random_state=0)#,C = 0.001)


df_stat = pd.DataFrame()
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

t00 = time.time()

#cv_results = cross_validate(model, X, y, cv=3, n_jobs=-1, return_estimator=True,  )
#cv_results = cross_val_predict(model, X, y, cv=3, n_jobs=-1)# , return_estimator=True,  )

y_sum = pd.Series(np.zeros_like(y), index = y.index)

n_blends = 10
for i,rs in enumerate(range(n_blends)):
    t0 = time.time()
    skf = StratifiedKFold(n_splits=2, shuffle = True, random_state = rs )
    y_pred = y.copy()
    list_folds_scores = []
    for train_index, test_index in skf.split(X, y):
        model.fit(X.iloc[train_index,:], y.iloc[train_index] )
        y_pred_on_test_fold = model.predict_proba(X.iloc[test_index,:])[:,1]
        y_pred.iloc[test_index] = y_pred_on_test_fold
        score_on_fold = log_loss(y.iloc[test_index], y_pred_on_test_fold, labels = [0,1] ) 
        list_folds_scores.append(score_on_fold)
    y_sum += y_pred 
    print( np.round( np.mean(cv_score) , 5 ) , np.round(loc_time,2),'seconds passed' )
    y_blend = y_sum / (i+1)
    cv_score =   log_loss(y, y_blend, labels = [0,1] )  
    df_stat.loc[rs,'Blended CV LogLoss'] = np.mean(cv_score)

    cv_score =   log_loss(y, y_pred, labels = [0,1] )  
    loc_time = time.time()-t0
    df_stat.loc[rs,'CV LogLoss'] = np.mean(cv_score)
    
    df_stat.loc[rs,'LogLoss Folds Mean'] = np.mean(cv_score)
    df_stat.loc[rs,'LogLoss Folds Mean'] = np.mean(cv_score)
    for t in range(len(list_folds_scores)): 
        df_stat.loc[rs,'LogLoss Fold'+str(t)] = list_folds_scores[t]
    
    print('Blend score: ', np.round( np.mean(cv_score) , 5 ), 'Average score', np.round( df_stat['CV LogLoss'].mean() , 5 )   ) # , np.round(total_time,2),'loc_time passed' )

y_blend = y_sum / n_blends
cv_score =   log_loss(y, y_blend, labels = [0,1] )  
print( np.mean(cv_score) ) # , np.round(total_time,2),'loc_time passed' )

total_time = time.time()-t00    


print(np.round(total_time,2),'seconds total passed')
print( df_stat.mean() )

list_c = ['Blended CV LogLoss', 'CV LogLoss']
fig = plt.figure(figsize = (16,5)); sp = 0
sp+=1;fig.add_subplot(1,2,sp)
for c in list_c[:1]:
    #df_stat[c].plot()
    plt.plot( df_stat[c], '*-' , label = c) 
plt.legend()

sp+=1;fig.add_subplot(1,2,sp)
for c in list_c:
    df_stat[c].plot()
for t in range(len(list_folds_scores)): 
    f = 'LogLoss Fold'+str(t)
    plt.plot( df_stat[f], '*-' , label = f ) 

plt.legend()
plt.show()

df_stat


In [ ]:
print(np.round(total_time,2),'seconds total passed')
print( df_stat.mean() )

list_c = ['Blended CV LogLoss', 'CV LogLoss']
fig = plt.figure(figsize = (16,5)); sp = 0
sp+=1;fig.add_subplot(1,2,sp)
for c in list_c[:1]:
    #df_stat[c].plot()
    plt.plot( df_stat[c], '*-' , label = c) 
plt.legend()

sp+=1;fig.add_subplot(1,2,sp)
for c in list_c:
    df_stat[c].plot()
for t in range(len(list_folds_scores)): 
    f = 'LogLoss Fold'+str(t)
    plt.plot( df_stat[f], '*-' , label = f ) 

plt.legend()
plt.show()

df_stat


# 3 Folds

In [ ]:
#sklearn.model_selection.cross_validate(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, 
# verbose=0, fit_params=None, pre_dispatch='2*n_jobs', return_train_score=False, return_estimator=False, error_score=nan)

n_splits = 3

model = LogisticRegression(random_state=0)#,C = 0.001)


df_stat = pd.DataFrame()
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

t00 = time.time()

#cv_results = cross_validate(model, X, y, cv=3, n_jobs=-1, return_estimator=True,  )
#cv_results = cross_val_predict(model, X, y, cv=3, n_jobs=-1)# , return_estimator=True,  )

y_sum = pd.Series(np.zeros_like(y), index = y.index)

n_blends = 10
for i,rs in enumerate(range(n_blends)):
    t0 = time.time()
    skf = StratifiedKFold(n_splits=n_splits, shuffle = True, random_state = rs )
    y_pred = y.copy()
    list_folds_scores = []
    for train_index, test_index in skf.split(X, y):
        model.fit(X.iloc[train_index,:], y.iloc[train_index] )
        y_pred_on_test_fold = model.predict_proba(X.iloc[test_index,:])[:,1]
        y_pred.iloc[test_index] = y_pred_on_test_fold
        score_on_fold = log_loss(y.iloc[test_index], y_pred_on_test_fold, labels = [0,1] ) 
        list_folds_scores.append(score_on_fold)
    y_sum += y_pred 
    print( np.round( np.mean(cv_score) , 5 ) , np.round(loc_time,2),'seconds passed' )
    y_blend = y_sum / (i+1)
    cv_score =   log_loss(y, y_blend, labels = [0,1] )  
    df_stat.loc[rs,'Blended CV LogLoss'] = np.mean(cv_score)

    cv_score =   log_loss(y, y_pred, labels = [0,1] )  
    loc_time = time.time()-t0
    df_stat.loc[rs,'CV LogLoss'] = np.mean(cv_score)
    
    df_stat.loc[rs,'LogLoss Folds Mean'] = np.mean(cv_score)
    df_stat.loc[rs,'LogLoss Folds Mean'] = np.mean(cv_score)
    for t in range(len(list_folds_scores)): 
        df_stat.loc[rs,'LogLoss Fold'+str(t)] = list_folds_scores[t]
    
    print('Blend score: ', np.round( np.mean(cv_score) , 5 ), 'Average score', np.round( df_stat['CV LogLoss'].mean() , 5 )   ) # , np.round(total_time,2),'loc_time passed' )

y_blend = y_sum / n_blends
cv_score =   log_loss(y, y_blend, labels = [0,1] )  
print( np.mean(cv_score) ) # , np.round(total_time,2),'loc_time passed' )

total_time = time.time()-t00    


print(np.round(total_time,2),'seconds total passed')
print( df_stat.mean() )

list_c = ['Blended CV LogLoss', 'CV LogLoss']
fig = plt.figure(figsize = (16,5)); sp = 0
sp+=1;fig.add_subplot(1,2,sp)
for c in list_c[:1]:
    #df_stat[c].plot()
    plt.plot( df_stat[c], '*-' , label = c) 
plt.legend()

sp+=1;fig.add_subplot(1,2,sp)
for c in list_c:
    df_stat[c].plot()
for t in range(len(list_folds_scores)): 
    f = 'LogLoss Fold'+str(t)
    plt.plot( df_stat[f], '*-' , label = f ) 

plt.legend()
plt.show()

df_stat


In [ ]:
print(np.round(total_time,2),'seconds total passed')
print( df_stat.mean() )

list_c = ['Blended CV LogLoss', 'CV LogLoss']
fig = plt.figure(figsize = (16,5)); sp = 0
sp+=1;fig.add_subplot(1,2,sp)
for c in list_c[:1]:
    #df_stat[c].plot()
    plt.plot( df_stat[c], '*-' , label = c) 
plt.legend()

sp+=1;fig.add_subplot(1,2,sp)
for c in list_c:
    df_stat[c].plot()
for t in range(len(list_folds_scores)): 
    f = 'LogLoss Fold'+str(t)
    plt.plot( df_stat[f], '*-' , label = f ) 

plt.legend()
plt.show()

df_stat


# 5 Folds

In [ ]:
#sklearn.model_selection.cross_validate(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, 
# verbose=0, fit_params=None, pre_dispatch='2*n_jobs', return_train_score=False, return_estimator=False, error_score=nan)

n_splits = 5

model = LogisticRegression(random_state=0)#,C = 0.001)


df_stat = pd.DataFrame()
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

t00 = time.time()

#cv_results = cross_validate(model, X, y, cv=3, n_jobs=-1, return_estimator=True,  )
#cv_results = cross_val_predict(model, X, y, cv=3, n_jobs=-1)# , return_estimator=True,  )

y_sum = pd.Series(np.zeros_like(y), index = y.index)

n_blends = 10
for i,rs in enumerate(range(n_blends)):
    t0 = time.time()
    skf = StratifiedKFold(n_splits=n_splits, shuffle = True, random_state = rs )
    y_pred = y.copy()
    list_folds_scores = []
    for train_index, test_index in skf.split(X, y):
        model.fit(X.iloc[train_index,:], y.iloc[train_index] )
        y_pred_on_test_fold = model.predict_proba(X.iloc[test_index,:])[:,1]
        y_pred.iloc[test_index] = y_pred_on_test_fold
        score_on_fold = log_loss(y.iloc[test_index], y_pred_on_test_fold, labels = [0,1] ) 
        list_folds_scores.append(score_on_fold)
    y_sum += y_pred 
    print( np.round( np.mean(cv_score) , 5 ) , np.round(loc_time,2),'seconds passed' )
    y_blend = y_sum / (i+1)
    cv_score =   log_loss(y, y_blend, labels = [0,1] )  
    df_stat.loc[rs,'Blended CV LogLoss'] = np.mean(cv_score)

    cv_score =   log_loss(y, y_pred, labels = [0,1] )  
    loc_time = time.time()-t0
    df_stat.loc[rs,'CV LogLoss'] = np.mean(cv_score)
    
    df_stat.loc[rs,'LogLoss Folds Mean'] = np.mean(cv_score)
    df_stat.loc[rs,'LogLoss Folds Mean'] = np.mean(cv_score)
    for t in range(len(list_folds_scores)): 
        df_stat.loc[rs,'LogLoss Fold'+str(t)] = list_folds_scores[t]
    
    print('Blend score: ', np.round( np.mean(cv_score) , 5 ), 'Average score', np.round( df_stat['CV LogLoss'].mean() , 5 )   ) # , np.round(total_time,2),'loc_time passed' )

y_blend = y_sum / n_blends
cv_score =   log_loss(y, y_blend, labels = [0,1] )  
print( np.mean(cv_score) ) # , np.round(total_time,2),'loc_time passed' )

total_time = time.time()-t00    


print(np.round(total_time,2),'seconds total passed')
print( df_stat.mean() )

list_c = ['Blended CV LogLoss', 'CV LogLoss']
fig = plt.figure(figsize = (16,5)); sp = 0
sp+=1;fig.add_subplot(1,2,sp)
for c in list_c[:1]:
    #df_stat[c].plot()
    plt.plot( df_stat[c], '*-' , label = c) 
plt.legend()

sp+=1;fig.add_subplot(1,2,sp)
for c in list_c:
    df_stat[c].plot()
for t in range(len(list_folds_scores)): 
    f = 'LogLoss Fold'+str(t)
    plt.plot( df_stat[f], '*-' , label = f ) 

plt.legend()
plt.show()

df_stat


In [ ]:
print(np.round(total_time,2),'seconds total passed')
print( df_stat.mean() )

list_c = ['Blended CV LogLoss', 'CV LogLoss']
fig = plt.figure(figsize = (16,5)); sp = 0
sp+=1;fig.add_subplot(1,2,sp)
for c in list_c[:1]:
    #df_stat[c].plot()
    plt.plot( df_stat[c], '*-' , label = c) 
plt.legend()

sp+=1;fig.add_subplot(1,2,sp)
for c in list_c:
    df_stat[c].plot()
for t in range(len(list_folds_scores)): 
    f = 'LogLoss Fold'+str(t)
    plt.plot( df_stat[f], '*-' , label = f ) 

plt.legend()
plt.show()

df_stat


# 10 Folds

In [ ]:
#sklearn.model_selection.cross_validate(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, 
# verbose=0, fit_params=None, pre_dispatch='2*n_jobs', return_train_score=False, return_estimator=False, error_score=nan)

n_splits = 10

model = LogisticRegression(random_state=0)#,C = 0.001)


df_stat = pd.DataFrame()
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

t00 = time.time()

#cv_results = cross_validate(model, X, y, cv=3, n_jobs=-1, return_estimator=True,  )
#cv_results = cross_val_predict(model, X, y, cv=3, n_jobs=-1)# , return_estimator=True,  )

y_sum = pd.Series(np.zeros_like(y), index = y.index)

n_blends = 10
for i,rs in enumerate(range(n_blends)):
    t0 = time.time()
    skf = StratifiedKFold(n_splits=n_splits, shuffle = True, random_state = rs )
    y_pred = y.copy()
    list_folds_scores = []
    for train_index, test_index in skf.split(X, y):
        model.fit(X.iloc[train_index,:], y.iloc[train_index] )
        y_pred_on_test_fold = model.predict_proba(X.iloc[test_index,:])[:,1]
        y_pred.iloc[test_index] = y_pred_on_test_fold
        score_on_fold = log_loss(y.iloc[test_index], y_pred_on_test_fold, labels = [0,1] ) 
        list_folds_scores.append(score_on_fold)
    y_sum += y_pred 
    print( np.round( np.mean(cv_score) , 5 ) , np.round(loc_time,2),'seconds passed' )
    y_blend = y_sum / (i+1)
    cv_score =   log_loss(y, y_blend, labels = [0,1] )  
    df_stat.loc[rs,'Blended CV LogLoss'] = np.mean(cv_score)

    cv_score =   log_loss(y, y_pred, labels = [0,1] )  
    loc_time = time.time()-t0
    df_stat.loc[rs,'CV LogLoss'] = np.mean(cv_score)
    
    df_stat.loc[rs,'LogLoss Folds Mean'] = np.mean(cv_score)
    df_stat.loc[rs,'LogLoss Folds Mean'] = np.mean(cv_score)
    for t in range(len(list_folds_scores)): 
        df_stat.loc[rs,'LogLoss Fold'+str(t)] = list_folds_scores[t]
    
    print('Blend score: ', np.round( np.mean(cv_score) , 5 ), 'Average score', np.round( df_stat['CV LogLoss'].mean() , 5 )   ) # , np.round(total_time,2),'loc_time passed' )

y_blend = y_sum / n_blends
cv_score =   log_loss(y, y_blend, labels = [0,1] )  
print( np.mean(cv_score) ) # , np.round(total_time,2),'loc_time passed' )

total_time = time.time()-t00    


print(np.round(total_time,2),'seconds total passed')
print( df_stat.mean() )

list_c = ['Blended CV LogLoss', 'CV LogLoss']
fig = plt.figure(figsize = (16,5)); sp = 0
sp+=1;fig.add_subplot(1,2,sp)
for c in list_c[:1]:
    #df_stat[c].plot()
    plt.plot( df_stat[c], '*-' , label = c) 
plt.legend()

sp+=1;fig.add_subplot(1,2,sp)
for c in list_c:
    df_stat[c].plot()
for t in range(len(list_folds_scores)): 
    f = 'LogLoss Fold'+str(t)
    plt.plot( df_stat[f], '*-' , label = f ) 

plt.legend()
plt.show()

df_stat


In [ ]:
print(np.round(total_time,2),'seconds total passed')
print( df_stat.mean() )

list_c = ['Blended CV LogLoss', 'CV LogLoss']
fig = plt.figure(figsize = (16,5)); sp = 0
sp+=1;fig.add_subplot(1,2,sp)
for c in list_c[:1]:
    #df_stat[c].plot()
    plt.plot( df_stat[c], '*-' , label = c) 
plt.legend()

sp+=1;fig.add_subplot(1,2,sp)
for c in list_c:
    df_stat[c].plot()
for t in range(len(list_folds_scores)): 
    f = 'LogLoss Fold'+str(t)
    plt.plot( df_stat[f], '*-' , label = f ) 

plt.legend()
plt.show()

df_stat
